In [1]:
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings

warnings.filterwarnings("ignore")
dataview_folder = 'stockdata/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

/home/xinger/anaconda3/envs/IIA/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Dataview loaded successfully.


In [2]:
dv.add_formula('divert', '- Correlation(vwap_adj, volume, 10)', is_quarterly=False)
dv.add_formula('divert2', '- Correlation(vwap_adj, volume, 20)', is_quarterly=False)
factor_dict = {
    "signal1":dv.get_ts('divert'),
    "signal2":dv.get_ts('divert2')
}

# 因子处理方法

In [3]:
signal = factor_dict["signal1"]
signal.head()

symbol,000001.SZ,000002.SZ,000008.SZ,000009.SZ,000024.SZ,000027.SZ,000039.SZ,000046.SZ,000060.SZ,000061.SZ,...,601997.SH,601998.SH,603000.SH,603160.SH,603288.SH,603799.SH,603833.SH,603858.SH,603885.SH,603993.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20160104,-0.516941,-inf,-0.717011,-0.381988,NaN,-0.540330,-0.414171,-0.696174,-0.712166,-0.830877,...,NaN,-0.670815,-0.532163,NaN,-0.574892,-0.403839,NaN,NaN,0.100549,-0.398122
20160105,-0.288026,-inf,0.022007,-0.245200,NaN,-0.206120,-0.392205,-0.307085,-0.392649,-0.654043,...,NaN,-0.526281,-0.230478,NaN,0.180595,0.210184,NaN,NaN,0.294341,0.214205
20160106,-0.337502,-inf,0.074163,-0.364918,NaN,-0.287856,-0.512486,-0.415161,0.172436,-0.639272,...,NaN,-0.494766,-0.311435,NaN,0.579645,0.321342,NaN,NaN,0.361880,0.378866
20160107,-0.422917,-inf,-0.228013,-0.596544,NaN,-0.293643,-0.687983,-0.636234,-0.002605,-0.665710,...,NaN,-0.513661,-0.545025,NaN,-0.019905,-0.052134,NaN,NaN,-0.108086,0.278405
20160108,-0.122549,-inf,0.008911,-0.522839,NaN,0.057260,-0.566546,-0.296512,-0.002037,-0.403560,...,NaN,-0.146023,-0.424566,NaN,0.291232,0.173483,NaN,NaN,-0.165425,0.396511


In [4]:
from jaqs.research import process

In [7]:
signal = process.winsorize(signal) #去极值
signal = process.standardize(signal) #ｚ－ｓｃｏｒｅ标准化
signal = process.rank_standardize(signal) #因子在截面排序并归一化到０－１
signal = process.get_disturbed_factor(signal) #给每个因子加一个很小的绕动
signal = process.neutralize(signal,
                            factorIsMV=False,#是否是市值类因子
                            group_field="sw1",#行业分类类型　"sw1", "sw1", "sw1", "sw1", "zz1", "zz2"
                            dv=None,#dataview
                            ds=None) #data_api #行业市值中性化


Begin: DataApi login 18566262672@tcp://data.tushare.org:8910
    Already login as 18566262672, skip init_from_config
Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
    At fields 
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query groups (industry)...
Data has been successfully prepared.


In [8]:
signal.head()

,000001.SZ,000008.SZ,000009.SZ,000027.SZ,000039.SZ,000046.SZ,000060.SZ,000063.SZ,000069.SZ,000156.SZ,...,601985.SH,601988.SH,601991.SH,601992.SH,601998.SH,603000.SH,603288.SH,603799.SH,603885.SH,603993.SH
20160104,0.057155,-0.269168,0.354600,0.020556,0.106169,0.002996,-0.253366,0.103221,-0.102065,-0.165223,...,-0.043790,0.023363,-0.040433,0.213179,-0.243647,0.137902,0.033385,0.260293,0.393033,0.271383
20160105,0.031009,0.124174,0.188679,-0.140815,-0.347608,0.071802,-0.254188,0.027831,-0.022818,0.101057,...,0.262529,0.294361,0.221186,0.083580,-0.182940,-0.089358,0.413169,0.262245,0.336349,0.232999
20160106,0.021157,0.012853,0.136832,-0.193775,-0.386515,-0.027571,0.032391,-0.036258,-0.033866,0.252998,...,0.220260,0.240010,0.271975,0.209249,-0.121093,-0.147120,0.500973,0.164455,0.308314,0.115478
20160107,0.020804,-0.031314,0.050716,-0.040790,-0.434261,-0.210312,0.085192,-0.069390,0.095726,0.292825,...,0.119608,0.333255,0.233025,0.110888,-0.094466,-0.192343,0.440557,0.118936,0.200704,0.171162
20160108,0.023018,0.026355,0.031407,0.001412,-0.384246,-0.095714,-0.034549,-0.123892,0.068406,0.263247,...,0.055925,0.333665,0.180491,0.165875,-0.027008,-0.187112,0.468469,0.215547,0.002258,0.223517


# 多因子方法

In [9]:
from jaqs.research import multi_factor

In [10]:
# 因子间存在较强同质性时，使用施密特正交化方法对因子做正交化处理，用得到的正交化残差作为因子
new_factors = multi_factor.orthogonalize(factors_dict=factor_dict,
                           standardize_type="rank",#输入因子标准化方法，有"rank"（排序标准化）,"z_score"(z-score标准化)两种（"rank"/"z_score"）
                           winsorization=False) #是否对输入因子去极值
                        

In [11]:
new_factors

{'signal1':           000001.SZ  000002.SZ  000008.SZ  000009.SZ  000027.SZ  000039.SZ  \
 20160104   0.427885        NaN   0.132212   0.718750   0.521635   0.723558   
 20160105   0.302885        NaN   0.406250   0.430288   0.521635   0.343750   
 20160106   0.295673        NaN   0.425481   0.336538   0.435096   0.283654   
 20160107   0.310096        NaN   0.367788   0.211538   0.490385   0.283654   
 20160108   0.365385        NaN   0.367788   0.168269   0.555288   0.153846   
 20160111   0.358173        NaN   0.382212   0.302885   0.581731   0.250000   
 20160112   0.490385        NaN   0.514423   0.512019   0.377404   0.641827   
 20160113   0.360577        NaN   0.430288   0.471154   0.394231   0.247596   
 20160114   0.420673        NaN   0.531250   0.514423   0.266827   0.173077   
 20160115   0.338942        NaN   0.711538   0.487981   0.286058   0.221154   
 20160118   0.396635        NaN   0.819712   0.620192   0.281250   0.228365   
 20160119   0.500000        NaN   0.03846

In [12]:
#  多因子组合-等权重
factor = multi_factor.combine_factors(factor_dict,
                                      standardize_type="rank",
                                      winsorization=False,
                                      weighted_method="equal_weight")

In [14]:
factor.head()

symbol,000001.SZ,000002.SZ,000008.SZ,000009.SZ,000024.SZ,000027.SZ,000039.SZ,000046.SZ,000060.SZ,000061.SZ,...,601997.SH,601998.SH,603000.SH,603160.SH,603288.SH,603799.SH,603833.SH,603858.SH,603885.SH,603993.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20160104,0.426859,NaN,0.115108,0.731415,NaN,0.553957,0.745803,0.129496,0.184652,0.474820,...,NaN,0.254197,0.383693,NaN,0.489209,0.714628,NaN,NaN,0.827338,0.484412
20160105,0.287770,NaN,0.383693,0.436451,NaN,0.532374,0.347722,0.220624,0.218225,0.374101,...,NaN,0.143885,0.395683,NaN,0.688249,0.618705,NaN,NaN,0.800959,0.568345
20160106,0.290168,NaN,0.381295,0.347722,NaN,0.472422,0.314149,0.165468,0.477218,0.328537,...,NaN,0.187050,0.345324,NaN,0.817746,0.594724,NaN,NaN,0.798561,0.793765
20160107,0.290168,NaN,0.342926,0.213429,NaN,0.491607,0.321343,0.110312,0.489209,0.258993,...,NaN,0.187050,0.184652,NaN,0.726619,0.534772,NaN,NaN,0.781775,0.860911
20160108,0.354916,NaN,0.352518,0.177458,NaN,0.551559,0.177458,0.333333,0.340528,0.237410,...,NaN,0.285372,0.175060,NaN,0.817746,0.508393,NaN,NaN,0.599520,0.858513


In [18]:
def mask_index_member():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = ~(df_index_member >0) #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
    # 跌停
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

mask = mask_index_member()
can_enter,can_exit = limit_up_down()
price = dv.get_ts("close_adj")

In [19]:
#  多因子组合-动态加权　最大化ＩＲ
max_IR_props = {
    'price': price,
    'benchmark_price': None,# 为空计算的是绝对收益　不为空计算相对收益
    'period': 5,
    'mask': mask,
    'can_enter': can_enter,
    'can_exit': can_exit,
    'forward': True,
    'commission': 0.0008,
    "covariance_type": "simple",  #协方差矩阵估算方法 还可以为"shrink" 
    "rollback_period": 120} #用多少期的ic做权重计算

In [20]:
factor = multi_factor.combine_factors(factor_dict,
                                      standardize_type="rank",
                                      winsorization=False,
                                      weighted_method="max_IR",
                                      max_IR_props=max_IR_props)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 68%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 68%


In [21]:
factor.tail()

symbol,000001.SZ,000002.SZ,000008.SZ,000009.SZ,000024.SZ,000027.SZ,000039.SZ,000046.SZ,000060.SZ,000061.SZ,...,601997.SH,601998.SH,603000.SH,603160.SH,603288.SH,603799.SH,603833.SH,603858.SH,603885.SH,603993.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20171225,0.520384,0.400480,0.357314,0.731415,NaN,0.009592,0.417266,0.225420,0.290168,0.244604,...,0.767386,0.534772,0.882494,0.338129,0.913669,0.105516,0.820144,0.702638,0.251799,0.071942
20171226,0.549161,0.553957,0.206235,0.654676,NaN,0.021583,0.472422,0.175060,0.390887,0.127098,...,0.793765,0.484412,0.923261,0.395683,0.863309,0.189448,0.681055,0.834532,0.410072,0.141487
20171227,0.573141,0.597122,0.081535,0.438849,NaN,0.028777,0.529976,0.537170,0.189448,0.191847,...,0.688249,0.613909,0.896882,0.383693,0.858513,0.333333,0.769784,0.822542,0.247002,0.345324
20171228,0.575540,0.585132,0.105516,0.616307,NaN,0.057554,0.541966,0.561151,0.076739,0.230216,...,0.520384,0.688249,0.875300,0.342926,0.815348,0.335731,0.750600,0.884892,0.311751,0.141487
20171229,0.635492,0.601918,0.069544,0.625899,NaN,0.026379,0.398082,0.609113,0.083933,0.266187,...,0.817746,0.484412,0.812950,0.494005,0.637890,0.462830,0.709832,0.805755,0.251799,0.179856
